In [28]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import re

In [29]:
buildingpermits_pin_apn_df = pd.read_csv("buildingpermits_pin_apn.csv")
buildingpermits_pin_apn_df['downloaded'] = 0
buildingpermits_pin_apn_df

,PERMIT_NBR,PIN_NBR,APN,downloaded
0,21014-20000-04453,189B125 914,2120013026,0
1,19010-70000-06089,120B193 1436,5041022020,0
2,19010-10000-04757,112-5A207 49,5115011028,0
3,22010-20000-01092,183B089 509,2028035002,0
4,20016-90000-18359,171B101 414,2169016024,0
...,...,...,...,...
781511,19016-40000-05120,129A209 259,5144010***,0
781512,19016-30000-23532,129B121 111,4419001019,0
781513,09016-30001-14411,123B177 851,5043002018,0
781514,16030-20000-06017,201B125 122,2763037032,0


In [30]:
pin = "171B117    17"

In [31]:
def make_html_endpoint(pin):
  pin_parts = pin.split()
  if len(pin_parts) == 2:
    pin1, pin2 = pin_parts[0], pin_parts[1]

    spaces_in_between = 0
    for i, letter in enumerate(pin):
      if letter.isspace():
        spaces_in_between += 1

    space = "%20" * spaces_in_between


  return f'https://zimas.lacity.org/map.aspx?pin={pin1}{space}{pin2}&ajax=yes'

def get_html_response(html_endpoint):
  return requests.get(html_endpoint).text

html_endpoint = make_html_endpoint(pin)
html_response = get_html_response(html_endpoint)

In [32]:
html_response = """
<html><head><meta http-equiv=Content-Type" content="text/html; charset=windows-1252"></head><body>
"""+ html_response
html_response = html_response + "</body></html>"

# start_index = html_response.find('divTab5: "')
# end_index = html_response.find('", divTab6: "', start_index)
# if start_index != -1 and end_index != -1:
#     html_response = html_response[:start_index] + html_response[end_index + 13:]



html_response = html_response.replace("&nbsp;", " ").replace("&lt;", "<").replace("&gt;", ">")
    
# Fix tags by removing extra backslashes
html_response = re.sub(r"\\>", ">", html_response)
html_response = re.sub(r"\\<", "<", html_response)
    
# Format the HTML content with proper indentation
formatted_content = ""
indentation_level = 0
for line in html_response.splitlines():
  stripped_line = line.strip()
  if stripped_line.startswith("</"):
    indentation_level -= 1
  formatted_content += "    " * indentation_level + stripped_line + "\n"
  if stripped_line.startswith("<") and not stripped_line.startswith("</") and not stripped_line.endswith("/>"):
    indentation_level += 1


soup = BeautifulSoup(html_response, 'html.parser')



rows = soup.find_all('tr')

def find_text_for_label(label):
    for row in rows:
        cells = row.find_all('td')
        if cells and label in cells[0].get_text(strip=True):
            return cells[1].get_text(strip=True)
    return None



def find_addresses():
    addresses = []
    for row in rows:
        cells = row.find_all('td')
        if cells and 'Site Address' in cells[0].get_text(strip=True):
            addresses.append(cells[1].get_text(strip=True))
        return addresses


def find_multiple_text_for_label(label):
    cases = []
    # div_tab5 = soup.find(text=lambda text: text and 'divTab5:' in text)
    for row in rows:
        cells = row.find_all('td')
        if cells and label in cells[0].get_text(strip=True):
            if len(cells) <= 1:
                break
            case = cells[1].get_text(strip=True)
            cases.append(case)
    return cases


addresses = find_addresses()
zip_code = find_text_for_label('ZIP Code')
pin_number = find_text_for_label('PIN Number')
assessor_parcel_no = find_text_for_label('Assessor Parcel No. (APN)')
community_plan_area = find_text_for_label('Community Plan Area')
area_planning_commission = find_text_for_label('Area Planning Commission')
neighborhood_council = find_text_for_label('Neighborhood Council')
census_tract = find_text_for_label('Census Tract #')
zoning = find_text_for_label('Zoning')
general = find_text_for_label('Zoning')
general_plan_land_use = find_text_for_label('General Plan Land Use')
hillside_area = find_text_for_label('Hillside Area (Zoning Code)')
apn_area = find_text_for_label('APN Area (Co. Public Works)*')
rent_stabilization_ordinance = find_text_for_label('Rent Stabilization Ordinance (RSO)')
ellis_act_property = find_text_for_label('Ellis Act Property')
tenant_protection_act = find_text_for_label('AB 1482: Tenant Protection Act')
environmental_cases = find_multiple_text_for_label('Environmental')
city_planning_commission_cases = find_multiple_text_for_label('City Planning Commission')


print(addresses)
print(zip_code)
print(pin_number)
print(assessor_parcel_no)
print(community_plan_area)
print(area_planning_commission)
print(neighborhood_council)
print(census_tract)
print(zoning)
print(hillside_area)
print(apn_area)
print(rent_stabilization_ordinance)
print(ellis_act_property)
print(tenant_protection_act)
print(environmental_cases)
print(city_planning_commission_cases)

['19224 W LINNET ST']
91356
171B117    17
2163011004
Encino - Tarzana
South Valley
Tarzana
1394.02
RA-1
No
0.680 (ac)
No [APN: 2163011004]
No
See Notes
['ENV-2019-1743-EIR', 'ENV-2005-8253-ND']
['CPC-9708', 'CPC-2019-1745-CPU']
